✅25. Indicar el nombre y el tamaño de las aplicaciones educativas. (⭐)


In [ ]:
!pip install pyspark
!apt update

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=744d4a6bbc81cdfd4f852131fa754dd55213ffff35778cec2dca94f1b78f6cbd
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-sec

In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [ ]:
sqlContext = SQLContext(sc)
# Atencion, utilizo estos metodos para separar bien la forma en la que se lee del csv, ya que sino se lee de manera incorrecta en algunos casoa
app_data = sqlContext.read.option("delimiter", ",").option("escape", '"').csv('/content/drive/MyDrive/aa-OrgaDeDatos/googleplaystore.csv', header=True, inferSchema=True)
rdd_app_data = app_data.rdd

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [ ]:
# Hago esto para poder "droppear" las repetidas. Use este metodo porque distinct elimina la fila si y solo si todos los campos de dos filas coinciden, cosa que
# no pasa en este caso. Hay apps que tienen todos los campos iguales y difieren por algunas reviews, lo que puede afectar al resultado final en este caso.
def get_key(x):
    return "{0}".format(x["App"])
rdd_app_data_filtered = rdd_app_data.map(lambda x: (get_key(x), x)).reduceByKey(lambda x,y: x)

La app que elimino es una app que tiene un problema al utilizar el take, al ser arabe se muestra mal, mixea el nombre y el size, por lo que decido eliminarla

In [10]:
education_rows = rdd_app_data_filtered.filter(lambda row: row[1]["Category"] == "EDUCATION" and row[1]["App"] != "Flame - درب عقلك يوميا").map(lambda row: (row[1]["App"], row[1]["Size"]))
education_rows.take(20)

# Utilizo el [1] para indexar previo al valor de la row por como quedo definido para droppear las filas repetidas previamente el rdd

[('Duolingo: Learn Languages Free', 'Varies with device'),
 ('TED', '18M'),
 ('English Communication - Learn English for Chinese (Learn English for Chinese)',
  '18M'),
 ('Khan Academy', '21M'),
 ('Learn English with Wlingua', '3.3M'),
 ('Ai La Trieu Phu - ALTP Free', '24M'),
 ('Princess Coloring Book', '39M'),
 ('Learn Spanish - Español', '3.2M'),
 ('English Grammar Test', '5.1M'),
 ('Speed Reading', '11M'),
 ('English for beginners', '27M'),
 ('Mermaids', 'Varies with device'),
 ('Learn Japanese, Korean, Chinese Offline & Free', '26M'),
 ('Kids Mode', '11M'),
 ('PBS KIDS Video', 'Varies with device'),
 ('Dinosaurs Coloring Pages', '41M'),
 ('Cars Coloring Pages', '49M'),
 ('Babbel – Learn Languages', '21M'),
 ('Math Tricks', '8.1M'),
 ('Monster Truck Driver & Racing', '51M')]